In [1]:
import pickle
import torch
from torch.utils.data import TensorDataset,random_split
import plotly.express as px
def get(name):
    d=[]
    with open(f'/kaggle/input/activations/{name}.pt','rb') as file:
        d=pickle.load(file)
    return d
values=[]
def operate(tensor,k=20):
    U=tensor-tensor.mean(dim=1,keepdim=True)
    U=U/torch.norm(U,dim=1,keepdim=True)
    _,s,V=torch.linalg.svd(U)
    values.append(s)
    return V[:k]
def get_errors(set1,set2):
    U=torch.matmul(set1,set1.T)
    V=torch.matmul(set1,set2.T)
    X=torch.linalg.solve(U,V)
    pos=torch.matmul(set1.T,X)
    return set2.T-pos
def overall(set1,set2):
    x=get_errors(set1,set2)
    y=torch.norm(x,dim=0)
    return y.mean().item()
def split(x_ds,y_ds):
    x=[]
    y=[]
    for i in range(len(x_ds)):
        x.append(x_ds[i][0])
    for i in range(len(y_ds)):
        y.append(y_ds[i][0])
    return torch.stack(x),torch.stack(y)
halu=get('halu_eval')
simple=get('simple_qa')
truth=get('truthful_qa')

In [2]:
def var(s,k):
    s1=s[k].item()
    s2=torch.norm(s).item()
    return (s1/s2)**2

In [3]:
halu_correct=torch.stack([x['correct']-x['wrong'] for x in halu])
simple_correct=torch.stack([x['correct'] for x in simple])
truth_correct=torch.stack([x['correct'][0][0,-1]-x['wrong'][0][0,-1] for x in truth])

d_h=TensorDataset(halu_correct)
d_s=TensorDataset(simple_correct)
d_t=TensorDataset(truth_correct)

H1,H2=split(*random_split(d_h,[0.5,0.5]))
S1,S2=split(*random_split(d_s,[0.5,0.5]))
H=halu_correct
S=simple_correct
T=truth_correct

In [4]:
values=[]
y1=operate(H,k=20)
y2=operate(T,k=2000)
e=torch.norm(get_errors(y1,y2),dim=0)

In [36]:
import math
variances0=[var(values[0],i) for i in range(len(values[0]))]
variances1=[var(values[1],i) for i in range(len(values[1]))]
v0=[sum(variances0[:i+1]) for i in range(len(variances0))]#[:len(variances1)]
#v1=[sum(variances1[:i+1]) for i in range(len(variances1))]
v2=[math.exp(-15/(x+11)) for x in range(len(v0))]
px.line({
    '1':v0,
#    '2':v1,
    '3':v2
})

In [ ]:
px.line(e.tolist())

In [ ]:
x=[H,H1,H2,S,S1,S2,T]
x=[operate(i,k=20) for i in x]
y=['HaluEval','HaluEval_split1','HaluEval_split2','SimpleQA','SimpleQA_split1','SimpleQA_split2','TruthfulQA']

data=[]
for a in x:
    data.append([])
    for b in x:
        data[-1].append(overall(a,b))
px.imshow(data,x=y,y=y,text_auto='.3f')

In [ ]:
px.line(torch.norm(get_errors(x[3],x[0]),dim=0).tolist())

In [ ]:
px.line(torch.norm(get_errors(x[3],x[0]),dim=0).tolist())

In [ ]:
a=get_errors(x[-1],x[0])

In [ ]:
sorted(torch.norm(a,dim=0).tolist())

In [ ]:
S1=[]
def null_space(A, rcond=None):
    u, s, vh = torch.linalg.svd(A, full_matrices=True)
    S1.append(s)
    M, N = u.shape[0], vh.shape[1]
    if rcond is None:
        rcond = torch.finfo(s.dtype).eps * max(M, N)
    tol = torch.max(s) * rcond
    num = torch.sum(s > tol, dtype=int)
    Q = vh[num:,:].T.conj()
    return Q

In [ ]:
null_space(torch.cat([x[0],x[3]]),rcond=0.15).shape

In [ ]:
px.line(S1[-1].tolist())